In [ ]:
!pip install torch==1.7.1
!pip install nltk==3.4.5
!pip install colorama==0.4.4
!pip install transformers==3.4.0
!pip install torchtext==0.3.1
!pip install pandas

In [ ]:
import pandas as pd
import random

print("Generating prefixes")

df = pd.read_csv('/kaggle/input/pplm-project/dataset_preprocessed.tsv', header=None, sep='\t', on_bad_lines='skip')
df = df[1].astype(str)

rand_prefix = []
for tweet in df.sample(n=500, replace=True).values.tolist():
    rand_prefix.append(("positive", " ".join(tweet.split()[:min(random.randint(3,5),len(tweet.split()))])))
for tweet in df.sample(n=500, replace=True).values.tolist():
    rand_prefix.append(("negative", " ".join(tweet.split()[:min(random.randint(3,5),len(tweet.split()))])))

pos_neg_dict = {
    "positive": 0,
    "negative": 1
}

print("{num_prefixes} prefixes have been generated".format(num_prefixes=len(rand_prefix)))

In [ ]:
import random
import csv
import joblib

generated_tweets = []
for i in range(0,len(rand_prefix)):
    print("{iteration}\tGenerating {sentiment} tweet with prefix '{prefix}'".format(
        iteration=i,
        sentiment=rand_prefix[i][0],
        prefix=rand_prefix[i][1]
    ))
    output = !python \
        /kaggle/input/pplm-project/run_pplm.py \
        --discrim generic \
        --discrim_weights /kaggle/input/pplm-project/discriminators/generic_classifier_head_epoch_5.pt \
        --discrim_meta /kaggle/input/pplm-project/discriminators/generic_classifier_head_meta.json \
        --class_label {pos_neg_dict[rand_prefix[i][0]]} \
        --cond_text "{rand_prefix[i][1]}" \
        --length 40 \
        --gamma 1.0 \
        --num_iterations 3 \
        --num_samples 1 \
        --stepsize 0.04 \
        --kl_scale 0.01 \
        --gm_scale 0.95 \
        --verbosity quiet
    generated_tweets.append((rand_prefix[i][0], rand_prefix[i][1] , " ".join(output).split("= Perturbed generated text 1 = <|endoftext|>")[1].strip()))
random.shuffle(generated_tweets)

print("Writing output to file for automatic evaluation")
with open("custom_discriminator_generated_tweets.tsv","w+") as output_file:
    outputWriter = csv.writer(output_file, delimiter='\t')
    outputWriter.writerow(["input_sentiment", "prefix", "generated_tweet"])
    outputWriter.writerows(generated_tweets)

random.shuffle(generated_tweets)
print("Writing output to file for manual evaluation")
with open("custom_discriminator_generated_tweets_MANUAL.tsv","w+") as output_file:
    outputWriter = csv.writer(output_file, delimiter='\t')
    outputWriter.writerow(["input_sentiment", "prefix", "generated_tweet"])
    outputWriter.writerows(generated_tweets[0:150])

print("############ COMPLETE ############")